In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import playerdashptshots
from nba_api.stats.endpoints import leaguedashplayerptshot
pd.options.mode.chained_assignment = None

data_DIR = "../fdata/"

# Each Dash Type Separately

In [ ]:
league, league_id = "NBA", "00"
season_start = 2023
season_end = 2024
seasons = np.arange(season_start, season_end, 1).astype(str)
dash_types = ["overall","shot_type","shot_clock","dribble","closest_def","closest_def_10","touch_time"]
for season in tqdm(seasons):
    season_str = season + "-" + str(int(season)+1)[-2:]
    stats = playerdashptshots.PlayerDashPtShots(league_id = league_id,team_id = 0,player_id = 0,season=season_str).get_data_frames()
    for i,d in enumerate(dash_types):
        df = stats[i].drop(columns=["SORT_ORDER","FGA_FREQUENCY","FG2A_FREQUENCY","FG3A_FREQUENCY"])
        df.to_parquet(data_DIR + f"{league}_Shots_{season}_{d}.parquet")

# Bin Shots by Condition

In [ ]:
general_range = ['Catch and Shoot', 'Pull Ups', 'Less than 10 ft', 'Other']
shot_clock = ['24-22', '22-18 Very Early', '18-15 Early', '15-7 Average', '7-4 Late', '4-0 Very Late']
dribbles = ['0 Dribbles', '1 Dribble', '2 Dribbles', '3-6 Dribbles', '7+ Dribbles']
closest_def = ['0-2 Feet - Very Tight', '2-4 Feet - Tight', '4-6 Feet - Open', '6+ Feet - Wide Open']
touch_time = ['Touch < 2 Seconds', 'Touch 2-6 Seconds', 'Touch 6+ Seconds']

In [ ]:
league, league_id = "NBA", "00"
season_start = 2023
season_end = 2024
seasons = np.arange(season_start, season_end, 1).astype(str)

In [ ]:
for season in tqdm(seasons):
    season_str = season + "-" + str(int(season)+1)[-2:]
    n = 0
    for a,b,c in product(general_range,closest_def,touch_time):
        n+=1
    dfa = []
    for a,b,c in tqdm(product(general_range,closest_def,touch_time),total=n):
        for i in range(5):
            try:
                stats = leaguedashplayerptshot.LeagueDashPlayerPtShot(
                    league_id = league_id,season=season_str,
                    general_range_nullable = a,
                    close_def_dist_range_nullable= b,
                    touch_time_range_nullable = c, 
                    # dribble_range_nullable =  d,
                ).get_data_frames()
                df1 = stats[0]
                df1["general_range"] = a
                df1["closest_def"] = b
                df1["touch_time"] = c
                dfa.append(df1)
                break
            except Exception as error:
                print(error)
                time.sleep(0.6)
                continue
    dfa1 = [df2 for df2 in dfa if not df2.empty]
    df = pd.concat(dfa1)
    df.to_parquet(data_DIR + f"{league}_Shots_{season}_All.parquet")